In [24]:
import cobra
import pandas as pd
from custom_functions_scRBA import *

In [2]:
model = cobra.io.load_json_model('./models/cobraModel_constantStoichOnly_minFluxSum.json')
model.solver = 'cplex'

Using license file /home/hvdinh16/Workspace/Softwares/gurobi910/linux64/gurobi.lic
Academic license - for non-commercial use only - expires 2021-07-26


In [5]:
df_raw = pd.read_excel('../scRBA_new/scRAM_network_frompy.xlsx')

In [33]:
with open('../scRBA_new/species.txt') as f:
    species = f.read().split('\n')

In [35]:
mets = [i for i in species if i[:4] == 'MET-']
mets = list(set(mets))
print('No of metabolites:', len(mets))

No of metabolites: 1608


In [36]:
genes = []; prots = []
for i in df_raw[df_raw.type == 'protein'].index:
    prot = df_raw.id[i][7:]
    prots.append(prot)
    if '_' in prot:
        comp = prot.split('_')[-1]
        gene = prot[:-len(comp)-1]
    else:
        gene = prot
    genes.append(gene)
    
genes = list(set(genes))
prots = list(set(prots))
print('No of genes:', len(genes))
print('No of proteins:', len(prots))

No of genes: 1104
No of proteins: 1136


In [74]:
enzs = []
idx = df_raw[df_raw.type == 'metabolic'].index

rxns = {'enz_assoc': [], 'spont': [], 'dummy': []}
rxns_trans = {'enz_assoc': [], 'spont': [], 'dummy': []}
rxns_ex = []
rxns_base = []
rxns_trans_base = []

for i in idx:
    tag,rxn_base_id,rxn_dir,enz_id = extract_details_from_rxnid(df_raw.id[i])
    enzs.append(enz_id)
    
    if rxn_base_id[:3] == 'EX_':
        rxns_ex.append(df_raw.id[i])
    elif 't' not in rxn_base_id:
        rxns_base.append(rxn_base_id)
        if enz_id in ['DUMMYENZ_m', 'DUMMYENZ_c']:
            rxns['dummy'].append(df_raw.id[i])
        elif enz_id in ['SPONT']:
            rxns['spont'].append(df_raw.id[i])
        else:
            rxns['enz_assoc'].append(df_raw.id[i])
    else:
        rxns_trans_base.append(rxn_base_id)
        if enz_id in ['DUMMYENZ_m', 'DUMMYENZ_c']:
            rxns_trans['dummy'].append(df_raw.id[i])
        elif enz_id in ['SPONT']:
            rxns_trans['spont'].append(df_raw.id[i])
        else:
            rxns_trans['enz_assoc'].append(df_raw.id[i])
    
rxns['total'] = sum(rxns.values(), [])
rxns_trans['total'] = sum(rxns_trans.values(), [])

enzs = list(set(enzs))
print('No of enzymes:', len(enzs))
print('')
print('No of reactions:', len(idx))
print('')
print('Metabolic reactions')
print('No of metabolic reactions:', len(rxns['total']))
rxns_base = list(set(rxns_base))
print('No of metabolic reactions (without copies for isozyme and reverse direction):', len(rxns_base))
rxns = {k:list(set(v)) for k,v in rxns.items()}
Ntotal = len(rxns['total'])
n = float(len(rxns['enz_assoc']))
print('No of enz associated reactions: %.0f' % (n / Ntotal * 100) + '%')
n = float(len(rxns['spont']))
print('No of spontaneous reactions: %.0f' % (n / Ntotal * 100) + '%')
n = float(len(rxns['dummy']))
print('No of median-enzyme associated reactions: %.0f' % (n / Ntotal * 100) + '%')
print('')
print('Transport reactions')
print('No of transport reactions:', len(rxns_trans['total']))
rxns_trans_base = list(set(rxns_trans_base))
print('No of transport reactions (without copies for isozyme and reverse direction):', len(rxns_trans_base))
rxns_trans = {k:list(set(v)) for k,v in rxns_trans.items()}
Ntotal = len(rxns_trans['total'])
n = float(len(rxns_trans['enz_assoc']))
print('No of transporter associated reactions: %.0f' % (n / Ntotal * 100) + '%')
n = float(len(rxns_trans['spont']))
print('No of spontaneous reactions: %.0f' % (n / Ntotal * 100) + '%')
n = float(len(rxns_trans['dummy']))
print('No of median-enzyme associated reactions: %.0f' % (n / Ntotal * 100) + '%')
print('')
print('No of exchage reactions:', len(rxns_ex))
print('No of protein and ribosome synthesis reactions')
print('')
n1 = df_raw[df_raw.type == 'protein'].shape[0]
n2 = df_raw[df_raw.type == 'ribosome'].shape[0]
n3 = df_raw[df_raw.type == 'enzyme'].shape[0]
print('No of protein, enzyme, and ribosome synthesis reactions:', n1+n2+n3)

No of enzymes: 934

No of reactions: 3265

Metabolic reactions
No of metabolic reactions: 1623
No of metabolic reactions (without copies for isozyme and reverse direction): 1137
No of enz associated reactions: 93%
No of spontaneous reactions: 3%
No of median-enzyme associated reactions: 4%

Transport reactions
No of transport reactions: 1467
No of transport reactions (without copies for isozyme and reverse direction): 597
No of transporter associated reactions: 49%
No of spontaneous reactions: 51%
No of median-enzyme associated reactions: 0%

No of exchage reactions: 175
No of protein and ribosome synthesis reactions

No of protein, enzyme, and ribosome synthesis reactions: 2077


In [ ]:
n = df_raw[df_raw.type == 'biomass'].shape[0] + df_raw[df_raw.type == 'debug'].shape[0]
print('No of biomass diluted related:', n)

In [58]:
df_raw.id[i]

'RXN-PYDX5PS_c_FWD-SNZ3SNO1'

In [56]:
rxns['spont']

['SPONT']

In [38]:
[e for e in enzs if 'DUMMY' in e]

['DUMMYENZ_m', 'DUMMYENZ_c']

In [39]:
[e for e in enzs if 'SPONT' in e]

['SPONT']

In [40]:
enz_id

'SNZ3SNO1'

In [26]:
set(df_raw.type)

{'biomass', 'debug', 'enzyme', 'metabolic', 'protein', 'ribosome'}

In [25]:
df_raw[df_raw.type == 'metabolic']

,id,type,symbolic_coeffs,reaction
0,RXN-BTDDH_c_FWD-YAL060W,metabolic,mu/keff,MET-btd_c + MET-nad_c + mu/keff ENZ-YAL060W --...
1,RXN-BTDDH_c_REV-YAL060W,metabolic,mu/keff,MET-actn__R_c + MET-h_c + MET-nadh_c + mu/keff...
2,RXN-13GS_c_FWD-GSC2RHO1,metabolic,mu/keff,MET-udpg_c + mu/keff ENZ-GSC2RHO1 --> MET-13BD...
3,RXN-13GS_c_FWD-GSC1RHO1,metabolic,mu/keff,MET-udpg_c + mu/keff ENZ-GSC1RHO1 --> MET-13BD...
4,RXN-16GS_c_FWD-YPR159W,metabolic,mu/keff,MET-udpg_c + mu/keff ENZ-YPR159W --> MET-16BDg...
...,...,...,...,...
3260,RXN-AKGDH_m_FWD-KGDCPLX,metabolic,mu/keff,MET-akg_m + MET-coa_m + MET-nad_m + mu/keff EN...
3261,RXN-ETFOXRq6_m_FWD-CIR1AIM45,metabolic,mu/keff,MET-fadh2_m + MET-q6_m + mu/keff ENZ-CIR1AIM45...
3262,RXN-PYDX5PS_c_FWD-SNZ1SNO1,metabolic,mu/keff,MET-g3p_c + MET-gln__L_c + MET-r5p_c + mu/keff...
3263,RXN-PYDX5PS_c_FWD-SNZ2SNO1,metabolic,mu/keff,MET-g3p_c + MET-gln__L_c + MET-r5p_c + mu/keff...


In [27]:
df_raw[df_raw.type == 'ribosome']

,id,type,symbolic_coeffs,reaction
4200,RIBO-rrna18s_c,ribosome,NaN,483 MET-atp_c + 348 MET-ctp_c + 459 MET-gtp_c ...
4201,RIBO-rrna5s_c,ribosome,NaN,30 MET-atp_c + 30 MET-ctp_c + 33 MET-gtp_c + 2...
4202,RIBO-rrna58s_c,ribosome,NaN,41 MET-atp_c + 36 MET-ctp_c + 37 MET-gtp_c + 4...
4203,RIBO-rrna25s_c,ribosome,NaN,899 MET-atp_c + 662 MET-ctp_c + 966 MET-gtp_c ...
4204,RIBO-ribosome_c,ribosome,NaN,PRO-YMR116C + PRO-YGR214W + PRO-YOR293W + PRO-...
4205,RIBO-riboactive_c,ribosome,friboactive,MET-ribosome_c --> friboactive MET-riboactive_c


No of biomass diluted related: 15


In [29]:
df_raw[df_raw.type == 'biomass']

,id,type,symbolic_coeffs,reaction
5342,BIODIL-SINKDUMMY_c,biomass,NaN,PRO-DUMMYPRO_c -->
5343,BIODIL-SINKDUMMY_m,biomass,NaN,PRO-DUMMYPRO_m -->
5344,BIODIL-PRO1,biomass,NaN,PRO-preModeledProtBiom_m --> BIO-modeledProtBi...
5345,BIODIL-PRO2,biomass,NaN,PRO-preModeledProtBiom_c --> BIO-modeledProtBi...
5346,BIODIL-PRO3,biomass,NaN,PRO-preUnmodeledProtBiom --> BIO-modeledProtBi...
5347,BIODIL-PRO4,biomass,NaN,PRO-preModeledProtBiom_c --> BIO-modeledProtBi...
5348,BIODIL-PRO5,biomass,NaN,PRO-preUnmodeledProtBiom --> BIO-modeledProtBi...
5349,BIODIL-PRO6,biomass,NaN,PRO-preUnmodeledProtBiom --> BIO-unmodeledProt...
5350,BIODIL-RNA,biomass,NaN,0.233979 MET-atp_c + 0.227364 MET-ctp_c + 0.23...
5351,BIODIL-BIODILFERMENTATION,biomass,"PARAM-BIO-unmodeledProtBiom,PARAM-BIO-modeledP...",PARAM-BIO-unmodeledProtBiom BIO-unmodeledProtB...


In [28]:
df_raw[df_raw.type == 'debug']

,id,type,symbolic_coeffs,reaction
5353,DMRXNS-BIO-unmodeledProtBiom,debug,NaN,BIO-unmodeledProtBiom -->
5354,DMRXNS-BIO-modeledProtBiom_c,debug,NaN,BIO-modeledProtBiom_c -->
5355,DMRXNS-BIO-modeledProtBiom_m,debug,NaN,BIO-modeledProtBiom_m -->
5356,DMRXNS-BIO-rna_c,debug,NaN,BIO-rna_c -->


In [ ]:
df_raw[df_raw.type == 'metabolic']

In [21]:
len(genes)

1104

In [22]:
len(prots)

1136

In [6]:
df_raw

,id,type,symbolic_coeffs,reaction
0,RXN-BTDDH_c_FWD-YAL060W,metabolic,mu/keff,MET-btd_c + MET-nad_c + mu/keff ENZ-YAL060W --...
1,RXN-BTDDH_c_REV-YAL060W,metabolic,mu/keff,MET-actn__R_c + MET-h_c + MET-nadh_c + mu/keff...
2,RXN-13GS_c_FWD-GSC2RHO1,metabolic,mu/keff,MET-udpg_c + mu/keff ENZ-GSC2RHO1 --> MET-13BD...
3,RXN-13GS_c_FWD-GSC1RHO1,metabolic,mu/keff,MET-udpg_c + mu/keff ENZ-GSC1RHO1 --> MET-13BD...
4,RXN-16GS_c_FWD-YPR159W,metabolic,mu/keff,MET-udpg_c + mu/keff ENZ-YPR159W --> MET-16BDg...
...,...,...,...,...
5352,BIODIL-BIODILRESPIRATION,biomass,"PARAM-BIO-unmodeledProtBiom,PARAM-BIO-modeledP...",PARAM-BIO-unmodeledProtBiom BIO-unmodeledProtB...
5353,DMRXNS-BIO-unmodeledProtBiom,debug,NaN,BIO-unmodeledProtBiom -->
5354,DMRXNS-BIO-modeledProtBiom_c,debug,NaN,BIO-modeledProtBiom_c -->
5355,DMRXNS-BIO-modeledProtBiom_m,debug,NaN,BIO-modeledProtBiom_m -->
